In [1]:
import numpy
import re
import math
import glob
#from nb_model import NB_model 



In [2]:
vocab = []
hamwords = {}
hamdoc_count = 0
hamfiles = [file for file in glob.glob("./train/*ham*.txt")]

for file in hamfiles:
    hamdoc_count  = hamdoc_count+1
    doc = open(file, 'r').read().lower()
    words = [word for word in re.split('[^a-zA-Z]',doc)]
    for word in words:
        if word != '':
            try:
                hamwords[word] += 1
            except KeyError:
                hamwords[word] = 1

spamdoc_count = 0
spamwords = {}
spamfiles = [file for file in glob.glob("./train/*spam*.txt")]
for file in spamfiles:
    spamdoc_count = spamdoc_count+1
    doc = open(file, 'r').read().lower()
    words = [word for word in re.split('[^a-zA-Z]',doc)]
    for word in words:
        if word != '':
            try:
                spamwords[word] += 1
            except KeyError:
                spamwords[word] = 1
    
print(len(vocab))
print(len(spamwords))
print(len(hamwords))

#(vocab)

0
47560
20101


In [5]:
hamwords

{'return': 1061,
 'path': 1036,
 'exmh': 2309,
 'workers': 1077,
 'admin': 4062,
 'spamassassin': 2598,
 'taint': 2997,
 'org': 9006,
 'delivered': 1361,
 'to': 10861,
 'yyyy': 1023,
 'localhost': 7694,
 'netnoteinc': 2088,
 'com': 12108,
 'received': 7004,
 'from': 10200,
 'by': 7861,
 'phobos': 1820,
 'labs': 1073,
 'postfix': 1987,
 'with': 7655,
 'esmtp': 4520,
 'id': 7652,
 'c': 2049,
 'for': 7612,
 'jm': 3132,
 'wed': 1766,
 'aug': 5415,
 'edt': 1352,
 'imap': 1012,
 'fetchmail': 997,
 'single': 1044,
 'drop': 1012,
 'ist': 1163,
 'listman': 820,
 'dogma': 1303,
 'slashnull': 1303,
 'g': 1977,
 'lcxvz': 1,
 'jmason': 1139,
 'localdomain': 206,
 'redhat': 1076,
 'f': 1230,
 'a': 6621,
 'ea': 65,
 'int': 303,
 'mx': 513,
 'corp': 301,
 'd': 2011,
 'mail': 1730,
 'lcuqx': 1,
 'smtp': 813,
 'lcuqy': 1,
 'ratree': 6,
 'psu': 15,
 'ac': 130,
 'th': 44,
 'lcgnl': 1,
 'delta': 16,
 'cs': 45,
 'mu': 7,
 'oz': 48,
 'au': 84,
 'coe': 3,
 'lcuil': 1,
 'ict': 6,
 'munnari': 40,
 'lcu': 1,
 'w

In [6]:
class NB_model:
    
    def __init__(self,msg):
        self.spam_prob_dict ={}
        self.ham_prob_dict = {}
        self.prob_spam = 0
        self.prob_ham = 0
        self.tot_ham_words = 0
        self.tot_spam_words = 0
        print(msg)
        
    def train(self, hm_dict, sp_dict, hm_prob, sp_prob):
        vocab_set = []
        self.prob_spam = sp_prob
        self.prob_ham = hm_prob
        for key,value in hm_dict.items():
            self.tot_ham_words += value
            vocab_set.append(key)
        vocab_set = set(vocab_set)
    
        print(len(vocab_set))
        for key,value in spamwords.items():
            self.tot_spam_words += value
            vocab_set.add(key)
        print(len(vocab_set))
        self.vocab = list(vocab_set)
        print(len(self.vocab))
        file = open("./model.txt", 'w+')
    
        index = 1
        for word in sorted(self.vocab):
            try:
                hm_dict[word] += 0
            except KeyError:
                hm_dict[word] = 0
                
            try:
                sp_dict[word] += 0
            except KeyError:
                sp_dict[word] = 0
                
            prob_word_ham = (hm_dict[word]+0.5)/(self.tot_ham_words+len(self.vocab))
            self.ham_prob_dict[word] = prob_word_ham
            prob_word_spam = (sp_dict[word]+0.5)/(self.tot_spam_words+len(self.vocab))
            self.spam_prob_dict[word] = prob_word_spam
           # print("%d  %s  %d  %f  %d  %f"%(index, word, hm_dict[word], prob_word_ham, sp_dict[word], prob_word_spam))
            file.write("%d  %s  %d  %f  %d  %f\n"%(index, word, hm_dict[word], prob_word_ham, sp_dict[word], prob_word_spam))
            index +=1
            
        file.close()
        
    def test(self, words):
        score_ham = math.log10(self.prob_ham)
        score_spam = math.log10(self.prob_spam)
        for word in words:
            if word != '':
                try:
                    self.ham_prob_dict[word] += 0
                except KeyError:
                    self.ham_prob_dict[word] = 0.5/(self.tot_ham_words+len(self.vocab))
                    self.spam_prob_dict[word] = 0.5/(self.tot_spam_words+len(self.vocab))
                
                score_ham = score_ham + math.log10(self.ham_prob_dict[word])
                score_spam = score_spam + math.log10(self.spam_prob_dict[word])              
            
        if score_ham > score_spam:
            return("ham",score_ham,score_spam)
        else:
            return("spam",score_ham,score_spam)
        




In [7]:
prob_ham = (hamdoc_count/(hamdoc_count+spamdoc_count))
prob_spam = (spamdoc_count/(hamdoc_count+spamdoc_count))

model = NB_model("creating object")
model.train(hamwords,spamwords,prob_ham,prob_spam)

creating object
20101
59953
59953


In [10]:
index = 1
y_act = []
y_pred = []
res_file = open("./result.txt", 'w+')
ham_test_files = [file for file in glob.glob("./test/*ham*.txt")]

# positive = ham and negative = spam

tp = 0
tn = 0
fp = 0
fn = 0

for file in ham_test_files:
    doc = open(file, 'r').read().lower()
    words = [word for word in re.split('[^a-zA-Z]',doc)]
    (result,h_score,s_score) = model.test(words)
    y_act.append(0)
    if result == "ham":
        output = 'right'
        tp += 1
        y_pred.append(0)
    else:
        output = 'wrong'
        fn += 1
        y_pred.append(1)
    #print("%d  %s  %s  %f  %f  %s  %s"%(index, file, result, h_score, s_score, "ham", output))
    res_file.write("%d  %s  %s  %f  %f  %s  %s \n"%(index, file, result, h_score, s_score, "ham", output))
    index += 1
    
spam_test_files = [file for file in glob.glob("./test/*spam*.txt")]

for file in spam_test_files:
    doc = open(file, 'r', encoding='cp437').read().lower()
    words = [word for word in re.split('[^a-zA-Z]',doc)]
    (result,h_score,s_score) = model.test(words)
    y_act.append(1)
    if result == "ham":
        output = 'wrong'
        fp += 1
        y_pred.append(0)
    else:
        output = 'right'
        tn += 1
        y_pred.append(1)
    #print("%d  %s  %s  %f  %f  %s  %s"%(index, file, result, h_score, s_score, "spam", output))
    res_file.write("%d  %s  %s  %f  %f  %s  %s \n"%(index, file, result, h_score, s_score, "spam", output))
    index += 1
    
res_file.close()


In [13]:
def result(true_pos, true_neg, false_pos, false_neg):
    print("-------------------------------------------Evaluation--------------------------------------------")
    print("true_pos : "+ str(true_pos) +"\ntrue_neg : "+str(true_neg)+"\nfalse_pos : "+ str(false_pos) +"\nfalse_neg : "+ str(false_neg) +"\n")
    precision = true_pos/(true_pos + false_pos)
    recall = true_pos/(true_pos + false_neg)
    f1 = 2 * ((precision * recall)/(precision + recall))
    accuracy = (true_pos + true_neg)/(true_pos + false_neg + true_neg + false_pos)
    print("Precision : "+str(precision*100) + "\nRecall : "+str(recall*100)+"\nAccuracy : "+str(f)+"\nAccuracy : "+str(accuracy*100))

In [12]:
result(tp, tn, fp, fn)

-------------------------------------------Evaluation--------------------------------------------
true_pos : 395
true_neg : 334
false_pos : 66
false_neg : 5

Precision : 85.68329718004338
Recall : 98.75
Accuracy : 91.125
